In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

In [ ]:
%run othello_game.ipynb
%run othello_ai.ipynb

#### Ermitteln der Standardabweichung
Im Folgenden werden zunächst einige Datenpunkte gesammelt, indem in verschiedenen Spielzuständen jeweils eine tiefe, und eine flache Suche durchgeführt wird. Die Spielzustände werden durch zufälliges Ziehen erreicht.

In [ ]:
import csv
import os


def sample_probcut_values(num_games, shallow_depth, deep_depth):
    filename = f'probcut_dataset_{PROBCUT_SHALLOW_DEPTH}_{PROBCUT_DEEP_DEPTH}.csv'
    file_exists = os.path.isfile(filename)
    if file_exists:
        print('using existing dataset')
    else:
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file, delimiter=',')
            writer.writerow(('moves', 'shallow', 'deep'))
            for i in range(num_games):
                state = GameState()
                while not state.game_over:
                    ai_make_move(random_ai, state, 0, None)
                    shallow_value = alphabeta(
                        state, PROBCUT_SHALLOW_DEPTH, combined_heuristic, -math.inf, math.inf)
                    deep_value = alphabeta(
                        state, PROBCUT_DEEP_DEPTH, combined_heuristic, -math.inf, math.inf)
                    print(f'shallow: {shallow_value}, deep: {deep_value}')
                    writer.writerow(
                        (state.num_pieces, shallow_value, deep_value))
            file.close()

In [ ]:
sample_probcut_values(20, PROBCUT_SHALLOW_DEPTH, PROBCUT_DEEP_DEPTH)

Daten Laden

In [ ]:
import pandas
filename = 'probcut_dataset_2_4.csv'
df = pandas.read_csv(filename)
shallow = df['shallow']
deep = df['deep']

Berechnung der Standardabweichung nach Anzahl der Steine auf dem Spielfeld

In [ ]:
import numpy as np
import sklearn.metrics as skl
import sklearn.linear_model as lm
import matplotlib.pyplot as plt
import seaborn as sns
import math

shallow_np = np.array(shallow)
deep_np = np.array(deep)
moves = np.array(df['moves'])

x = []
y = []

for i in range(5, 45):
    shallow_c = shallow[moves == i]
    deep_c = deep[moves == i]
    variance = np.var(np.stack([shallow_c, deep_c], axis=1))
    explained_variance = skl.explained_variance_score(shallow_c, deep_c)
    x.append(i)
    y.append(math.sqrt(variance))

model = lm.LinearRegression()
model.fit(np.array(x).reshape(-1, 1), np.array(y))

plt.figure(figsize=(15, 10))
sns.set(style='whitegrid')
plt.scatter(x, y)
plt.axvline(x=0.0, c='k')
plt.axhline(y=0.0, c='k')
plt.plot(x, x * model.coef_ + model.intercept_)
plt.xlabel('number of disks on the board')
plt.ylabel('standard deviation')
plt.title('Standard Deviation')
plt.show()

In [ ]:
print('x *', model.coef_[0], '+', model.intercept_)

#### Daten Visualisieren

In [ ]:
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
import seaborn as sns
model = lm.LinearRegression()
model.fit(shallow.values.reshape(len(shallow), 1), deep)
plt.figure(figsize=(15, 10))
sns.set(style='whitegrid')
plt.scatter(shallow, deep)
plt.axvline(x=0.0, c='k')
plt.axhline(y=0.0, c='k')
plt.plot(shallow, shallow * model.coef_ + model.intercept_)
plt.xlabel('shallow search')
plt.ylabel('deep search')
plt.title('Probcut Values')
plt.show()

Varianz, erklärte Varianz und Standardabweichung berechnen

In [ ]:
import numpy as np
import sklearn.metrics as skl
variance = np.var(np.stack([shallow, deep], axis=1))
print(f'variance: {variance}')
explained_variance = skl.explained_variance_score(shallow, deep)
print(f'standard distribution: {np.sqrt(variance)}')
print(f'explained variance: {explained_variance}')